## Аналіз A/B тесту

Маємо проаналізувати дані A/B тесту в популярній [грі Cookie Cats](https://www.facebook.com/cookiecatsgame). Це класична гра-головоломка в стилі «з’єднай три», де гравець повинен з’єднати плитки одного кольору, щоб очистити дошку та виграти рівень. На дошці також зображені співаючі котики :)

Під час проходження гри гравці стикаються з воротами, які змушують їх чекати деякий час, перш ніж вони зможуть прогресувати або зробити покупку в додатку. У цьому блоці завдань ми проаналізуємо результати A/B тесту, коли перші ворота в Cookie Cats було переміщено з рівня 30 на рівень 40. Зокрема, ми проаналізуємо вплив на утримання (retention) гравців. Тобто хочемо зрозуміти чи переміщення воріт на 10 рівнів пізніше якимось чином вплинуло на те, що користувачі перестають грати в гру раніше чи пізніше з точки зору кількості їх днів з моменту встановлення гри.

Будемо працювати з даними з файлу `cookie_cats.csv`. Змінні в даних наступні:

- userid - унікальний номер, який ідентифікує кожного гравця.
- version - чи потрапив гравець в контрольну групу (gate_30 - ворота на 30 рівні) чи тестову групу (gate_40 - ворота на 40 рівні).
- sum_gamerounds - кількість ігрових раундів, зіграних гравцем протягом першого тижня після встановлення
- retention_1 - чи через 1 день після встановлення гравець повернувся і почав грати?
- retention_7 - чи через 7 днів після встановлення гравець повернувся і почав грати?

Коли гравець встановлював гру, його випадковим чином призначали до групи gate_30 або gate_40.

1. Зчитайте дані АВ тесту у змінну `df` та виведіть середнє значення показника показник `retention_7` (утримання на 7 день) по версіям гри. Сформулюйте гіпотезу: яка версія дає краще утримання через 7 днів після встановлення гри?

Оскільки у нас немає очікуваного ефекту, ми не можете заздалегідь розрахувати необхідний розмір вибірки (sample size). У таких випадках краще використовувати всі доступні дані, щоб отримати максимальну статистичну потужність (statistical power) і знизити ймовірність помилки другого роду (β-помилки).

In [40]:
# імпортуємо необхідні бідліотеки 
import numpy as np
import pandas as pd
import scipy.stats as stats

In [41]:
# зчитаємо дата фрейм 
df = pd.read_csv('cookie_cats.csv')
df.head()

,userid,version,sum_gamerounds,retention_1,retention_7
0,116,gate_30,3,False,False
1,337,gate_30,38,True,False
2,377,gate_40,165,True,False
3,483,gate_40,1,False,False
4,488,gate_40,179,True,True


In [42]:
# подівімояс на інформацію про нього
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 90189 entries, 0 to 90188
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   userid          90189 non-null  int64 
 1   version         90189 non-null  object
 2   sum_gamerounds  90189 non-null  int64 
 3   retention_1     90189 non-null  bool  
 4   retention_7     90189 non-null  bool  
dtypes: bool(2), int64(2), object(1)
memory usage: 2.2+ MB


In [43]:
# перевіримо чи є користувачі які відносяться до обох груп
session_counts = df['userid'].value_counts(ascending=False)
multi_users = session_counts[session_counts > 1].count()

print(f'Є {multi_users} користувачів, які відносяться до обох груп.')

Є 0 користувачів, які відносяться до обох груп.


In [44]:
# подівімося чи на дві рівні группи розділені наші корістувачі
gate_30_count = df['version'].value_counts().get('gate_30', 0)
gate_40_count = df['version'].value_counts().get('gate_40', 0)
print(f'Розмір вибірки для Н0 {gate_30_count } корістувачів.')
print(f'Розмір вибірки для Н1 {gate_40_count } корістувачів.')

Розмір вибірки для Н0 44700 корістувачів.
Розмір вибірки для Н1 45489 корістувачів.


In [45]:
# як бачимо вибірки нерівні, обчислимо різницю 
diff = gate_40_count - gate_30_count 
print(f'Вибірка Н0 на {diff} користувачів менша за вибірку Н1')

Вибірка Н0 на 789 користувачів менша за вибірку Н1


In [46]:
# для чистоти єксперименту зрівняємо вибірки:
df = df.drop(df[df['version'] == 'gate_40'].sample(n=diff, random_state=42).index)
df['version'].value_counts()

version
gate_30    44700
gate_40    44700
Name: count, dtype: int64

In [47]:
# порахуємо середнє значення показника retention_7 (утримання на 7 день) по версіям гри
retention_means = df.groupby("version")["retention_7"].mean()
print(retention_means)

version
gate_30    0.190201
gate_40    0.182103
Name: retention_7, dtype: float64


Сформулюємо нульову та альтернативну гіпотези:
Н0 - немає значущої різниці в утриманні на 7-й день між версіями gate_30 і gate_40.
Н1 - Утримання на 7-й день у версії gate_30 є вищим, ніж у версії gate_40, оскільки середнє значення retention_7 у gate_30 більше.

2. Перевірте з допомогою z-тесту аналогічно до прикладу в лекції, чи дає якась з версій гри кращий показник `retention_7` на рівні значущості 0.05. Обчисліть також довірчі інтервали для двох вибірок. Виведіть результат у форматі:
```
z statistic: ...
p-value: ...
Довірчий інтервал 95% для групи control: [..., ...]
Довірчий інтервал 95% для групи treatment: [..., ...]
```
де замість `...` - обчислені значення. В якості висновка дайте відповідь на два питання:  
    1. чи є статистична значущою різниця між поведінкою користувачів у різних версіях гри?   
    2. чи перетинаються довірчі інтервали утримання користувачів з різних версій гри? Про що це каже?  
    
Зверніть увагу, в такому і схожому завданнях ми використовуєм `proportion` Z-тест. Це тому що в нас залежна змінна має бінарне значення (повернеться аби ні користувач, чи клікне або ні користувач в інших ситуаціях - всього два можливих значення в змінної: 0/1, True/False ). Якщо б ми вимірювали скажімо чи є стат. значущою різниця між вагою чоловіків і жінок в певній вибірці, ми б використовувавли функцію `statsmodels.stats.ztest`, бо залежна змінна `вага` є неперервною (тип float, замість типу int чи bool і тільки двох можливих значень).

In [48]:
from statsmodels.stats.proportion import proportions_ztest, proportion_confint
control_results = df[df['version'] == 'gate_30']['retention_7']
treatment_results = df[df['version'] == 'gate_40']['retention_7']

In [49]:
n_con = control_results.count()
n_treat = treatment_results.count()
successes = [control_results.sum(), treatment_results.sum()]
nobs = [n_con, n_treat]

z_stat, pval = proportions_ztest(successes, nobs=nobs)
(lower_con, lower_treat), (upper_con, upper_treat) = proportion_confint(successes, nobs=nobs, alpha=0.05)

print(f'z statistic: {z_stat:.2f}')
print(f'p-value: {pval:.3f}')
print(f'Довірчий інтервал 95% для групи control: [{lower_con:.3f}, {upper_con:.3f}]')
print(f'Довірчий інтервал 95% для групи treatment: [{lower_treat:.3f}, {upper_treat:.3f}]')

z statistic: 3.11
p-value: 0.002
Довірчий інтервал 95% для групи control: [0.187, 0.194]
Довірчий інтервал 95% для групи treatment: [0.179, 0.186]


In [53]:
# Рівень значущості 5%
alpha = 0.05
if pval < alpha:
    print('Ми маємо статистично значущу різницю між поведінкою користувачів у різних версіях гри.')
else:
    print('Ми не маємо статистично значущої різниці між поведінкою користувачів у різних версіях гри.')

Ми маємо статистично значущу різницю між поведінкою користувачів у різних версіях гри.


Довірчі інтервали не не перетинаються. І це свідчить про те, що є статистично значуща різниця між групами.
Крім того, p-value нашого Z-тесту дорівнює 0.002, це ще раз підтверджує, що різниця між групами статистично значуща, оскільки p-value значно менше 0.05.
На основі всього сказаного ми відхиляємо гіпотезу Н0 і приймаємо альтернативну гіпотезу(Н1 - Утримання на 7-й день у версії gate_30 є вищим, ніж у версії gate_40)

3. Є ще один тип тестів, який використовується для бінарної метрики як от "зробить юзер дію, чи ні" - тест [**Хі-квадрат**](https://www.bmj.com/about-bmj/resources-readers/publications/statistics-square-one/8-chi-squared-tests) (ще одне [пояснення](https://www.scribbr.com/statistics/chi-square-tests/) тесту з прикладами). В нього інші гіпотези Н0 і Н1 на відміну від z- та t-тестів. А також цей тест можна використовувати, якщо в нас більше за 2 досліджувані групи, тобто в нас не А/В тест, а А/B/C/D, наприклад.  

В **z- та t-тестах** (які відрізняються тим, що ми в першому не знаємо дисперсію генеральної сукупності, але якщо в нас великий набір даних, то ці два тести дають дуже схожі результати) **ми перевіряємо, чи є різниця у середніх показниках по групам користувачів**.  

А в **тесті Хі-квадрат ми перевіряємо чи є звʼязок між групою користувача і тим, чи він зробить цікаву нам дію**. Це ніби дослідження одного і того самого, але дещо різними способами. Для перевірки, можна виконувати кілька тестів (особливо, якщо один дає якийсь непереконливий результат типу р-значення 0.07 - наче і fail to regect H0 на рівні стат значущості 5%, але цікаво, що скажуть інші тести), тож, зробимо і ми тест хі-квадрат та порівняємо його результат з z-тестом.

Про різницю між тестами можна почитати ще [тут](https://stats.stackexchange.com/a/178860) - це просто пояснення користувача стековерфлоу, але там розумні люди сидять.

Для проведення хі-квадрат тесту скористаємось функцією з `scipy.stats` `chi2_contingency` для обчислення статистики хі-квадрат і р-значення для перевірки конкретної гіпотези. У цю функцію вам треба передати таблицю 2х2: кількість випадків для кожної версії гри і значення `retention_7`.

**Задача**: виконайте тест хі-квадрат на рівні значущості 5% аби визначити, чи є залежність між версією гри та тим, чи зайде гравець на 7ий день після встановлення гри.
Тут гіпотези наступні
- Н0: значення retention_7 не залежить від версії гри
- Н1: є залежність між версією гри і значенням retention_7

Виведіть p-значення та зробіть висновок.


In [60]:
import pandas as pd
from scipy.stats import chi2_contingency

In [61]:
# Створюємо таблицю 2x2 для retention_7 і версії гри
contingency_table = pd.crosstab(df['version'], df['retention_7'])
contingency_table

retention_7,False,True
version,,
gate_30,36198,8502
gate_40,36560,8140


In [62]:
# Виконуємо тест хі-квадрат
chi2_stat, p_value, dof, expected = chi2_contingency(contingency_table)
print(f'p-значення: {p_value}')

p-значення: 0.0019225955112778372


In [63]:
# Перевірка гіпотези
alpha = 0.05
if p_value < alpha:
    print("Відхиляємо нульову гіпотезу, є залежність між версією гри та retention_7.")
else:
    print("Не відхиляємо нульову гіпотезу, значення retention_7 не залежить від версії гри.")

Відхиляємо нульову гіпотезу, є залежність між версією гри та retention_7.


Як бачимо що тест Хі-квадрат підтвердів наші вісновки про залежність між версією грі та retention_7, а z-тест показа цю залежність - утримання на 7-й день у версії gate_30 є вищим, ніж у версії gate_40